In [62]:
import pandas as pd
import numpy as np
import os
import timeit
 
start_time = timeit.default_timer() # 시작 시간 체크

min_dataV = 0
max_dataV = [0,0,0]

#19년 20년 월별 데이터 생성
data2019 = {}
data2020 = {}

def csv_to_dataFrame(year,dataY):
    global min_dataV,max_dataV
    year = str(year)
    
    for indx,csvF in enumerate(os.listdir('./seoul_subway_data/{}'.format(year))):
        #19년도 데이터 안에 이미 데이터가 있으면 넘어가고 없으면 기본값으로 추가
        if not 'data' + csvF[-10:-4] in dataY.keys():
            dataY.setdefault('data' + csvF[-10:-4])
        else:
            continue

        try:
            data = pd.read_csv('./seoul_subway_data/' + year + '/' + csvF,encoding = 'UTF-8',index_col=False)
        except Exception as e:
            data = pd.read_csv('./seoul_subway_data/' + year + '/' + csvF,encoding = 'EUC-KR',index_col=False)
        dataY['data' + csvF[-10:-4]] = data
        
        
    c_dataY = dataY.copy()
    #데이터에서 불필요한 데이터 조정
    for dataYM in c_dataY.keys():
        t_dataY = pd.DataFrame()
        if '역ID' in dataY[dataYM].columns:
            dataY[dataYM].drop('등록일자',axis=1,inplace = True)
            dataY[dataYM].rename(columns = {'하차총승객수' : '등록일자'},inplace = True)
            dataY[dataYM].rename(columns = {'승차총승객수' : '하차총승객수'},inplace = True)
            dataY[dataYM].rename(columns = {'역명' : '승차총승객수'},inplace = True)
            dataY[dataYM].rename(columns = {'역ID' : '역명'},inplace = True)
            
        dataY[dataYM]['유동인구수'] = dataY[dataYM]['승차총승객수'] + dataY[dataYM]['하차총승객수']
        
        
        newData_min = {'사용일자':'', '승차총승객수':min_dataV, '하차총승객수':min_dataV, '유동인구수':min_dataV}

        #20190101 20190102 ...20190131
        dateKeys = dataY[dataYM].groupby('사용일자').indices.keys()
        
        #일별 데이터프레임 생성
        for i in dateKeys:
            globals()['dataY[data{}]'.format(i)] = pd.DataFrame()
            
        #일별 데이터 프레임에 값 대입 dataY[data20190101 .. ]
        for i in dateKeys:
            #사용일자 datatype(float -> str)으로 변환
            dataY[dataYM] = dataY[dataYM].astype({'사용일자':str})
            #노선명 역명 사용일자를 기준으로 멀티 인덱싱 하면서 나머지 승하차 유동인구 수 합하기
            tmp_grouped_dataF = dataY[dataYM].groupby(['노선명','역명','사용일자']).sum()
            tmp_grouped_dataF = tmp_grouped_dataF.sort_index()
            MulTdataF = tmp_grouped_dataF.unstack()
            """
            MulTdataF 데이터프레임 형태
                                                    승차총승객수 ...           |          하차총승객수              | 유동인구수
            
                                  사용일자     20191201 20191202 ...         | 사용일자 20191201 20191202 ...
            --------------------------------------------------------------------------------------------------------
            노선명  3호선 | 역명        |가능
                                    |회기
                                    |...
            --------------------------------------------------------------------------------------------------------
                  2호선 | 역명        |가능
                                    |회기
                                    |...
            """
            
            #각 컬럼에 해당하는 사용일자 컬럼에 해당하는 값들을 넣음
            tmp_getOn = MulTdataF['승차총승객수']['{}'.format(i)]
            tmp_getOff = MulTdataF['하차총승객수']['{}'.format(i)]
            tmp_float_Pop = MulTdataF['유동인구수']['{}'.format(i)]
            tmp_getOn = tmp_getOn.to_frame()
            tmp_getOff = tmp_getOff.to_frame()
            tmp_float_Pop = tmp_float_Pop.to_frame()

            dataY['data{}'.format(i)] = pd.concat([tmp_getOn,tmp_getOff,tmp_float_Pop],axis = 1,ignore_index=True).rename(columns = {0:'승차총승객수',1:'하차총승객수',2:'유동인구수'})
            
            dataY['data{}'.format(i)] = dataY['data{}'.format(i)].reset_index()
            dataY['data{}'.format(i)]['사용일자'] = i
            #인구수는 최저 0으로 고정 최대는 데이터마다 다름
            #근데 이게 지금 작동이 안되네...
            dataY['data{}'.format(i)] = dataY['data{}'.format(i)].append(newData_min,ignore_index=True)
            
#             print(dataY['data{}'.format(i)])
            
            #같은 노선 갯수 구해서 전체 합에서 노선 갯수만큼 나눠주기
            b_c = dataY['data{}'.format(i)].groupby('역명').count()[['승차총승객수','하차총승객수','유동인구수']]
            b   = dataY['data{}'.format(i)].groupby('역명').sum()[['승차총승객수','하차총승객수','유동인구수']]
            #DataframeGroupby 객체에서 sum을 하던 divide를 하던 뭔 짓을 한번 더 하면 dataframe으로 변환됨
            #이때 b_c와b의 groupby기준이 역명으로 인덱싱 되어있으니까 인덱스 이름을 '역명'으로 꺼내고 reset_index
            k   = b.divide(b_c).rename_axis('역명').reset_index()
            k['사용일자'] = dataY['data{}'.format(i)]['사용일자']
            
            dataY['data{}'.format(i)] = k
            
            t_dataY = pd.concat([t_dataY,dataY['data{}'.format(i)]],axis = 0,ignore_index=True)
            
        dataY[dataYM] = t_dataY.groupby('역명').sum().rename_axis('역명').reset_index()
        dataY[dataYM]['사용일자'] = dataYM[4:]
            
#지도를 어떻게 표현할지에 대해 필요있을 수 있는 코드
#(모든 월별 지도의 최대 레전드 값을 고정을 시켜놓으면 전체적으로 볼때는 알아보기 쉬우나 특정 몇몇 지역들이 지나치게 흐려짐)
#(월별로 지도의 최대 레전드 값을 다르게 하면 색은 눈에 띄게 표현 가능하나 전체적인 지도를 볼 때는 계속해서 레전드의 최댓값을 확인할 필요가 잇음)
#         if max_dataV[0] < dataF[dataY]['승차총승객수'].max():
#             max_dataV[0] = dataF[dataY]['승차총승객수'].max()
        
#         if max_dataV[1] < dataF[dataY]['하차총승객수'].max():
#             max_dataV[1] = dataF[dataY]['하차총승객수'].max()
            
#         if max_dataV[2] < dataF[dataY]['유동인구수'].max():
#             max_dataV[2] = dataF[dataY]['유동인구수'].max()           

csv_to_dataFrame(2019,data2019)
csv_to_dataFrame(2020,data2020)

terminate_time = timeit.default_timer() # 종료 시간 체크  
 
print("%f초 걸렸습니다." % (terminate_time - start_time))

KeyboardInterrupt: 

<h1>크롤링으로 구 csv 만들기</h1>

In [ ]:
# from selenium import webdriver
# import time
# import os, csv

# option = webdriver.ChromeOptions()
# option.add_argument('headless')

# # os.path.abspath('chromedriver')

# driver = webdriver.Chrome(os.path.abspath('chromedriver'),options=option)
# driver.get('https://gits.gg.go.kr/gtdb/web/trafficDb/railRoad/TransitSWPass.do')

# time.sleep(3)

# def setRadio():
#     driver.find_element_by_xpath('//*[@id="radio1"]').click()

# def setSelect1(sel1):
#     driver.find_element_by_xpath("//select[@name='select1']/option[text()='%s']" %sel1).click()
    
# def setSelect2(sel2):
#     driver.find_element_by_xpath("//select[@name='select2']/option[text()='%s']" %sel2).click()

# #soup로 고정된 html 값에서 radio1 버튼 찾아와서 클릭하기
# setRadio()

# #rad 버튼 클릭후 동적으로 변경된 웹페이지에서 select1아이디 값 가져오기
# select1 = driver.find_element_by_id('select1')
# t_sel1 = select1.text.split('\n')
# sel1 = []
# for i,t in enumerate(t_sel1):
#     if not t.strip() == '':
#         sel1.append(t.strip())

# #시 - 역명 넣을 csv파일 생상(기존에 있는건 삭제)
# if os.path.exists('./seoul_subway_data/구_역명.csv'):
#     os.remove('./seoul_subway_data/구_역명.csv')

#encoding 설정 안해주면 mac은 되는데 window는 안될 때 있음 (조심)
# with open('./seoul_subway_data/구_역명.csv','w',encoding = 'utf-8') as csvFile:
#     writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])    
#     writer.writeheader()
    
# #sel1 값을 넣은 후 sel2값 넣고 검색 후 list값들을 가져온 다음 데이터 정리
# for i in sel1:
#     setSelect1(i)
    
#     sel2 = driver.find_element_by_id('select2')
#     t_sel2 = sel2.text.split('\n')
#     for a in range(len(t_sel2)):
#         t_sel2[a] = t_sel2[a].strip()
#         #전체 선택 혹은 비어있는 경우는 넘어가기 (행정 구역만 필요하니까)
#         if '전체선택' == t_sel2[a] or '' == t_sel2[a]:
#             continue
        
#         setSelect2(t_sel2[a])
#         driver.find_element_by_xpath('//*[@id = "search"]').click()
#         li_loc = (driver.find_element_by_id('selList')).text.split('\n')

#         for i in li_loc:
#             #가져온 list값은 매봉(1호선) 이런식으로 돼있으므로 ()를 기준으로 역이름과 호선명 나눠주기
#             st_idx = i.find('(') + 1
#             f_idx = i.find(')')
#             t_sel2[a] = t_sel2[a].replace('_','')
#             #csv파일에 바로 넣기
#             with open('./seoul_subway_data/구_역명.csv','a',encoding = 'utf-8') as csvFile:
#                 writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])
#                 writer.writerow({'구':t_sel2[a],'역명':i[:st_idx-1]})
                
# driver.quit()

<h1>데이터 정리 최종</h1>

In [3]:
gu_sta_name = pd.read_csv('./seoul_subway_data/구_역명.csv',encoding='utf-8')

def insert_gu(dataY):
    #데이터에 구 추가
    for dataYMD in dataY.keys():
        if not '구' in dataY[dataYMD].columns:    
            dataY[dataYMD] = pd.merge(dataY[dataYMD],gu_sta_name,on='역명',how = 'inner')

            if True in dataY[dataYMD].duplicated():
                dataY[dataYMD] = dataY[dataYMD].drop_duplicates()
                dataY[dataYMD].reset_index(inplace=True,drop = True)

        dataY[dataYMD] = dataY[dataYMD].sort_values(['사용일자','역명'],ascending = True).reset_index(drop = True)             
    
insert_gu(data2019)
insert_gu(data2020)

DATA = {}
if not 'data2019' in DATA.keys():
    DATA['data2019'] = data2019
if not 'data2020' in DATA.keys():
    DATA['data2020'] = data2020

<h1>그냥 바로 해버리면 전국에 칠해지니까 서울지역으로 json 범위 축소 </h1><br><p>근데 왜 한번에 안되고 여러번에 거쳐야 되는지는 모르겠음...</p>

In [59]:
import json
import folium
import webbrowser
import time
import matplotlib.pyplot as plt

geo_path = './map/skorea-municipalities-2018-geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

#json 데이터 안다듬어주면 전국단위로 난리남
print('json 처리 전 지역 수 : ',len(geo_str['features']))
min_len_geoStr = False
k = [-1,-2]
#서울 지역 코드 값 = 1로 시작
#왜인지는 모르겠는데 한번에 처리가 안되서 이전 카운트 개수하고 현재 카운트 개수에 차이가 없을 때 break
while min_len_geoStr == False:
    if k[0] == k[1]:
        min_len_geoStr = True
    k[0] = len(geo_str['features'])
    for i,t in enumerate(geo_str['features']):
        if not t['properties']['code'][0] == '1':
            del geo_str['features'][i]
    k[1] = len(geo_str['features'])        

print('json 처리 후 지역 수 : ',len(geo_str['features']))

class subwayData:
    global geo_str,DATA
    
    """
     --------------------
    |    Map Function    |
     --------------------
    """ 

    def __init__(self,dataY : pd.DataFrame,ym : str):      
        #map 설정
        self.map=folium.Map(location=[37.5502, 126.982], zoom_start=11, min_zoom=11,
                    tiles='stamentoner')
        
        #font 설정
        plt.rcParams['font.family'] = 'Apple Gothic'        
        
        self.dataY      = dataY            #data2019     dtypes : dataframe
        self.ym         = ym               #201910
        self.dataYM     = 'data' + ym      #data201910   dtypes : str
        
        try:
            self.dataY[self.dataYM]
        except:
            print('No data')
            return None
        
        self.DrawGuMap()

    #지하철로 구 유동인구 파악
    def DrawGuMap(self):
        #맵 그려주기
        folium.Choropleth(
            geo_data=geo_str,
            data=self.dataY[self.dataYM], 
            columns=['구','유동인구수'], 
            key_on='feature.properties.name',
            fill_color='PuRd',
            legend_name='구별 지하철 유동인구 수'
        ).add_to(self.map)
        self.SaveMap()
    
    #지하철 역별 유동인구 파악
    def DrawSubMap(self):
        #맵 그려주기
        folium.Choropleth(
            geo_data=geo_str,
            data=self.dataY[self.dataYM], 
            columns=['역명','유동인구수'], 
            key_on='feature.properties.name',
            fill_color='PuRd',
            legend_name='구별 지하철 유동인구 수'
        ).add_to(self.map)
        self.SaveMap()
    
    #그린 맵 파일 확인 후 저장 (나중에 html파일을 웹에 올려야 되서 따로 저장이 필요하긴함)
    def SaveMap(self):
        #쥬피터에서 보여주면 한글 깨져서 동일 경로 map 디렉토리에 해당 맵을 만들어줌
        if os.path.exists('./map'):
            if os.path.exists('./map/{}.html'.format(self.ym)):
                os.remove('./map/{}.html'.format(self.ym))
        else:
            os.makedirs('./map')
        self.map.save('./map/{}.html'.format(self.ym))    
        
    #파일 삭제
    def DelMap(self):
        if os.path.exists('./map/{}.html'.format(self.ym)):
            os.remove('./map/{}.html'.format(self.ym))        

    #파일 열기 
    def OpenMap(self):
        try:
            if os.path.exists('./map/{}.html'.format(self.ym)):
                BASE_DIR = os.path.dirname(os.path.abspath('map/{}.html'.format(self.ym)))
                webbrowser.open('file:' + os.path.join(BASE_DIR,'{}.html'.format(self.ym)))
                #파일 삭제전에 잠시 텀을 줘서 저장한게 보이게끔 함
                #나중에 파일 삭제 관련해서 다시 다뤄야할 때를 대비 (이 코드 안에서 삭제를 없애야 될 수도 있음 / 그래야 html에서 가져올수 있으니까)
                time.sleep(1)
                self.DelMap()
            else:
                raise Exception(self.ym + ' file is not exist')
        except Exception as e:
            print(e)
            return False
    """
     ----------------------
    |    Graph Function    |
     ----------------------
    """        
        
        
    #graph를 하나로 묶고 subplots을 쓸지 나눠서 각각의 fig를 만들고 html에서 div를 통해서 구분을 지어줄지 결정해야 됨
    def DrawBarGraph(self):
        #임시로 사이즈는 대충 둠
        print(self.dataY[self.dataYM])
        plt.figure(figsize = (20,20))
        plt.bar(self.dataY[self.dataYM]['역명'],self.dataY[self.dataYM])
        plt.xticks(rotation = 90)
        plt.titles('{} 유동인구 데이터'.format(dataYM))
        plt.show()
    
    def DrawLineGraph(self):
        pass
        

# def setSubwayData(dataY,ymd):
#     mySub_data = subwayData(dataY,ymd)
#     return mySub_data


def setSubwayData(params):
    global DATA
    dataY = DATA['data{}'.format(params['user_input_date']['yy'])]
    
    if not params['user_input_date']['st'] == '' and not params['user_input_date']['ed'] == '':
        ymd = params['user_input_date']['yy'] + params['user_input_date']['mm'] + params['user_input_date']['st']        
        st_sub_data = subwayData(dataY,ymd)
        
        ymd = params['user_input_date']['yy'] + params['user_input_date']['mm'] + params['user_input_date']['ed']                
        ed_sub_data = subwayData(dataY,ymd)        
        return st_sub_data,ed_sub_data
    
    else:
        ymd = params['user_input_date']['yy'] + params['user_input_date']['mm']
        mySub_data = subwayData(dataY,ymd)
        return mySub_data









json 처리 전 지역 수 :  250
json 처리 후 지역 수 :  25


<h1>클라이언트 딴에서 실행</h1>

In [64]:
#DATA = server와 연동해서 server의 DATA값을 가져와야 함

#user로부터 date입력 받기 (필수)     default = yyyy-mm input_format = yyyy mm stdd eddd
#user로부터 구 입력 받기   (선택)    default = 서울시 전체 input_format = gu_name 
#user로부터 역명 입력 받기  (선택)    default = 전체 역 input_format = station_name


if __name__ == '__main__':
    global DATA
    exit_flag = 0
    
    params = {
        'user_input_date'     :'',
        'user_gu_name'        :'',
        'user_station_name'   :''
    }
    
    while exit_flag == 0:
        #######################입력 란        #######################
        #나중엔 js에서 인풋값으로 사용자한테 받아와야 함
        try:
            a = input('Input yyyy mm | yyyy mm start_date end_date (ex:2020 01 | 2020 01 01 31) : ')
            b = a.split(' ')
            user_input_date = {
                'yy' : '',
                'mm' : '',
                'st' : '',
                'ed' : ''
            }
            for i,t in enumerate(b):
                user_input_date[list(user_input_date.keys())[i]] = t
            
            gu_name = input('input gu name : ')
            st_name = input('input station name : ')            
            
            params = {
                'user_input_date' : user_input_date,
                'user_gu_name' : gu_name,
                'user_station_name' : st_name
            }
        except Exception as e:
            print(e)
            
        

            
        #######################실행 란        #######################
            
        try:
            #일 값을 받을 경우 추가
            if not user_input_date['st'] == '' and not user_input_date['ed'] == '':                
                st_ym = user_input_date['yy'] + user_input_date['mm'] + user_input_date['st']
                ed_ym = user_input_date['yy'] + user_input_date['mm'] + user_input_date['ed']            
                
            #유저로부터 받은 연도가 DATA.keys()에 없으면 다음 확인
            for datayear in DATA.keys():
                if not user_input_date['yy'] in datayear:
                    continue
                #유저로부터 둘다 받았을 때만 확인
                if not user_input_date['st'] == '' and not user_input_date['ed'] == '':
                    st_my_subway_data,ed_my_subway_data = setSubwayData(params) 
                    
                    if not st_my_subway_data.OpenMap() == False and not ed_my_subway_data.OpenMap() == False:
                        exit_flag = 1
                        break
                    

                else:
                    my_subway_data = setSubwayData(params)
                
                    #map이 열리면 break
                    if not my_subway_data.OpenMap() == False:
                        exit_flag = 1
                        break
                
                #안열리면 입력 형식에 문제있음
                print("{} isn't correct year".format(user_input_date['yy']))
        except Exception as e:
            print(e)
            break




Input yyyy mm | yyyy mm start_date end_date (ex:2020 01 | 2020 01 01 31) : 2010
input gu name : 
input station name : 
Input yyyy mm | yyyy mm start_date end_date (ex:2020 01 | 2020 01 01 31) : 2020
input gu name : 
input station name : 
No data
2020 file is not exist
2020 isn't correct year


KeyboardInterrupt: Interrupted by user